In [2]:
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
pd.set_option('display.max_columns', None)

# Como se expandiu o AL ao longo do tempo?

In [2]:
rnal = pd.read_csv('data/csvs/rnal.csv')

In [3]:
gdf = gpd.read_file('data/geojson/porto.geojson')
polygon = gdf.geometry[0]

def is_inside(row):
    point = Point(row['X'], row['Y'])
    return polygon.contains(point)

rnal['is_inside'] = rnal.apply(is_inside, axis=1)
rnal = rnal[rnal['is_inside'] == True].drop("is_inside", axis=1)

In [5]:
type_map = {'Apartamento':1,'Moradia':2,'EstabelecimentoHospedagem':3,'EstabelecimentoHospedagemHostel':4,'Quartos':5}
freg_map = {'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória':1,'Bonfim':2,'União das freguesias de Lordelo do Ouro e Massarelos':3,'Paranhos':4,'União das freguesias de Aldoar, Foz do Douro e Nevogilde':5,'Campanhã':6,'Ramalde':7}

start_date = datetime(2011, 1, 1) # AL licenses start in 2011
end_date = datetime(2023, 12, 31)

al_json = []

for idx, single_al in rnal.sort_values(by='DataAberturaPublico').reset_index().iterrows():
    al_date = datetime.strptime(single_al.DataAberturaPublico[:-3], '%Y/%m/%d %H:%M:%S')
    al_entry = {}
    al_entry["id"] = idx+1
    al_entry["m"] = al_date.strftime('%m')
    al_entry["y"] = al_date.strftime('%y')
    al_entry["ts"] = round(min(max((al_date - start_date).days / (end_date - start_date).days, 0), 1), 2)
    al_entry["type"] = type_map[single_al.Modalidade]
    al_entry["freg"] = freg_map[single_al.Freguesia]
    al_entry["coord"] = [round(single_al.X,6),round(single_al.Y,6)]
    al_json.append(al_entry)

with open(f'../web/public/static/data/al.json', 'w') as fp:
    json.dump(al_json, fp, separators=(',', ':'))

# Quais as zonas de maior pressão?

In [3]:
censos = pd.read_csv('../Censos/seccoes-final.csv')

In [9]:
freg_map = {'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória':1,'Bonfim':2,'União das freguesias de Lordelo do Ouro e Massarelos':3,'Paranhos':4,'União das freguesias de Aldoar, Foz do Douro e Nevogilde':5,'Campanhã':6,'Ramalde':7}

censos_json = []

for idx, single_block in censos.reset_index().iterrows():
    censos_entry = {}
    censos_entry["geo"] = single_block.geometry
    censos_entry["als"] = single_block.ALs
    censos_entry["ratio"] = single_block.pressao
    censos_entry["freg"] = freg_map[single_block.Freguesia]
    censos_json.append(censos_entry)

with open(f'../web/public/static/data/censos.json', 'w') as fp:
    json.dump(censos_json, fp, separators=(',', ':'))

In [12]:
single_block.geometry

'MULTIPOLYGON (((-8.599050830471858 41.1704689632344, -8.599475270156416 41.170491995655034, -8.599391251759887 41.170993532906024, -8.599361103057639 41.17101183468953, -8.599324655382437 41.171020733530575, -8.598995758801557 41.17105866692871, -8.599050830471858 41.1704689632344)))'

In [4]:
freg_map = {'União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória':1,'Bonfim':2,'União das freguesias de Lordelo do Ouro e Massarelos':3,'Paranhos':4,'União das freguesias de Aldoar, Foz do Douro e Nevogilde':5,'Campanhã':6,'Ramalde':7}

censos

,Unnamed: 0,OBJECTID,BGRI2021,DT21,DTMN21,DTMNFR21,DTMNFRSEC21,SECNUM21,SSNUM21,SECSSNUM21,SUBSECCAO,NUTS1,NUTS2,NUTS3,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_CLASS_CONST_1_OU_2_ALOJ,N_EDIFICIOS_CLASS_CONST_3_OU_MAIS_ALOJAMENTOS,N_EDIFICIOS_EXCLUSIV_RESID,N_EDIFICIOS_1_OU_2_PISOS,N_EDIFICIOS_3_OU_MAIS_PISOS,N_EDIFICIOS_CONSTR_ANTES_1945,N_EDIFICIOS_CONSTR_1946_1980,N_EDIFICIOS_CONSTR_1981_2000,N_EDIFICIOS_CONSTR_2001_2010,N_EDIFICIOS_CONSTR_2011_2021,N_EDIFICIOS_COM_NECESSIDADES_REPARACAO,N_ALOJAMENTOS_TOTAL,N_ALOJAMENTOS_FAMILIARES,N_ALOJAMENTOS_FAM_CLASS_RHABITUAL,N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA,N_RHABITUAL_ACESSIVEL_CADEIRAS_RODAS,N_RHABITUAL_COM_ESTACIONAMENTO,N_RHABITUAL_PROP_OCUP,N_RHABITUAL_ARRENDADOS,N_AGREGADOS_DOMESTICOS_PRIVADOS,N_ADP_1_OU_2_PESSOAS,N_ADP_3_OU_MAIS_PESSOAS,N_NUCLEOS_FAMILIARES,N_NUCLEOS_FAMILIARES_COM_FILHOS_TENDO_O_MAIS_NOVO_MENOS_DE_25,N_INDIVIDUOS,N_INDIVIDUOS_H,N_INDIVIDUOS_M,N_INDIVIDUOS_0_14,N_INDIVIDUOS_15_24,N_INDIVIDUOS_25_64,N_INDIVIDUOS_65_OU_MAIS,SHAPE_Length,SHAPE_Area,geometry,ALs,pressao,Freguesia,Freguesia2011
0,0,52,13121000702,13,1312,131210,131210007,7,2,702,13121000702,1,11,11A,59.0,44.0,15.0,58.0,43.0,16.0,25.0,26.0,5.0,3.0,0.0,34.0,202.0,202.0,155.0,47.0,29.0,73.0,103.0,39.0,155.0,105.0,50.0,98.0,40.0,328.0,145.0,183.0,29.0,34.0,184.0,81.0,612.308514,21064.473451,MULTIPOLYGON (((-8.619215409378157 41.17106299...,3.0,0.014851,Paranhos,PARANHOS
1,1,53,13120302118,13,1312,131203,131203021,21,18,2118,13120302118,1,11,11A,21.0,13.0,7.0,21.0,11.0,10.0,0.0,19.0,2.0,0.0,0.0,13.0,73.0,72.0,58.0,14.0,0.0,14.0,13.0,43.0,58.0,46.0,12.0,33.0,10.0,134.0,54.0,80.0,8.0,10.0,55.0,61.0,481.233389,13437.276410,MULTIPOLYGON (((-8.582126078716755 41.17132007...,0.0,0.000000,Campanhã,CAMPANHÃ
2,2,54,13121103402,13,1312,131211,131211034,34,2,3402,13121103402,1,11,11A,31.0,26.0,5.0,31.0,20.0,11.0,17.0,9.0,1.0,4.0,0.0,7.0,78.0,78.0,71.0,7.0,10.0,21.0,26.0,42.0,71.0,50.0,21.0,40.0,18.0,143.0,67.0,76.0,15.0,8.0,79.0,41.0,707.178859,13484.048739,MULTIPOLYGON (((-8.65034890692763 41.170070134...,0.0,0.000000,Ramalde,RAMALDE
3,3,55,13121600903,13,1312,131216,131216009,9,3,903,13121600903,1,11,11A,57.0,46.0,11.0,57.0,44.0,13.0,0.0,44.0,13.0,0.0,0.0,7.0,109.0,109.0,94.0,15.0,24.0,65.0,64.0,23.0,94.0,62.0,32.0,57.0,24.0,205.0,89.0,116.0,33.0,12.0,100.0,60.0,824.199641,24498.620921,MULTIPOLYGON (((-8.670053736515053 41.17092891...,0.0,0.000000,"União das freguesias de Aldoar, Foz do Douro e...",ALDOAR
4,4,56,13121600403,13,1312,131216,131216004,4,3,403,13121600403,1,11,11A,4.0,4.0,0.0,4.0,4.0,0.0,0.0,2.0,1.0,1.0,0.0,2.0,6.0,6.0,2.0,4.0,1.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,5.0,2.0,3.0,0.0,1.0,2.0,2.0,177.035284,1829.033862,MULTIPOLYGON (((-8.656888121332463 41.17093816...,0.0,0.000000,"União das freguesias de Aldoar, Foz do Douro e...",ALDOAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,1654,202893,13121100503,13,1312,131211,131211005,5,3,503,13121100503,1,11,11A,27.0,0.0,26.0,26.0,0.0,27.0,0.0,0.0,0.0,0.0,27.0,2.0,271.0,270.0,261.0,9.0,4.0,1.0,5.0,251.0,261.0,173.0,88.0,171.0,72.0,595.0,269.0,326.0,66.0,60.0,303.0,166.0,752.009082,26949.364452,MULTIPOLYGON (((-8.654939481754848 41.17139789...,0.0,0.000000,Ramalde,RAMALDE
1655,1655,202894,13121103406,13,1312,131211,131211034,34,6,3406,13121103406,1,11,11A,54.0,46.0,8.0,54.0,41.0,13.0,13.0,27.0,3.0,3.0,8.0,26.0,97.0,97.0,78.0,19.0,22.0,29.0,43.0,27.0,78.0,48.0,30.0,49.0,28.0,174.0,77.0,97.0,32.0,20.0,89.0,33.0,795.220212,19418.258481,MULTIPOLYGON (((-8.654962486714794 41.17313053...,0.0,0.000000,Ramalde,RAMALDE
1656,1656,202987,13121702001,13,1312,131217,131217020,20,1,2001,13121702001,1,11,11A,61.0,18.0,41.0,51.0,9.0,52.0,38.0,20.0,2.0,1.0,0.0,56.0,235.0,233.0,142.0,91.0,13.0,6.0,43.0,91.0,142.0,118.0,24.0,59.0,22.0,358.0,152.0,206.0,22.0,31.0,172.0,133.0,847.347386,40377.342191

In [14]:
from shapely.wkt import loads, dumps
from shapely.geometry import mapping, shape

def round_coordinates(geom, decimal_places=6):
    geojson = mapping(geom)
    def round_recursive(coord_list):
        if isinstance(coord_list[0], list):  # If the item is another list, recurse
            return [round_recursive(sublist) for sublist in coord_list]
        else:
            # Handling the rounding of each coordinate separately
            return [round(coord, decimal_places) for coord in coord_list]
    geojson['coordinates'] = round_recursive(geojson['coordinates'])
    return shape(geojson)

# Provided MULTIPOLYGON string
multipolygon_str = 'MULTIPOLYGON (((-8.599050830471858 41.1704689632344, -8.599475270156416 41.170491995655034, -8.599391251759887 41.170993532906024, -8.599361103057639 41.17101183468953, -8.599324655382437 41.171020733530575, -8.598995758801557 41.17105866692871, -8.599050830471858 41.1704689632344)))'

# Convert the string to a Shapely geometry
geom = loads(multipolygon_str)

# Round the coordinates
rounded_geom = round_coordinates(geom, decimal_places=6)

# Convert the Shapely geometry back to a string
rounded_str = dumps(rounded_geom)

# Print or utilize the rounded string
print(rounded_str)

TypeError: type tuple doesn't define __round__ method

In [16]:
mapping(geom)

{'type': 'MultiPolygon',
 'coordinates': [(((-8.599050830471858, 41.1704689632344),
    (-8.599475270156416, 41.170491995655034),
    (-8.599391251759887, 41.170993532906024),
    (-8.599361103057639, 41.17101183468953),
    (-8.599324655382437, 41.171020733530575),
    (-8.598995758801557, 41.17105866692871),
    (-8.599050830471858, 41.1704689632344)),)]}

# Hoteis

In [17]:
hotels = pd.read_csv('../Hoteis/hotels.csv')

In [28]:
hotels

,id,geometry,novos_ET
0,Via Norte,"{'type': 'MultiLineString', 'coordinates': [[[...",0
1,Via de Cintura Interna,"{'type': 'MultiLineString', 'coordinates': [[[...",0
2,Rua da Conceição,"{'type': 'MultiLineString', 'coordinates': [[[...",0
3,Rua de José Falcão,"{'type': 'MultiLineString', 'coordinates': [[[...",2
4,Rua do General Silveira,"{'type': 'MultiLineString', 'coordinates': [[[...",0
...,...,...,...
2054,Praça Artur Santos Silva,"{'type': 'MultiLineString', 'coordinates': [[[...",0
2055,Praceta Luís António Verney,"{'type': 'MultiLineString', 'coordinates': [[[...",0
2056,DriveThrough Burger King,"{'type': 'MultiLineString', 'coordinates': [[[...",0
2057,Rua de Manuel Marques,"{'type': 'MultiLineString', 'coordinates': [[[...",0


In [34]:
import json

geojson = {}
geojson["type"] = "FeatureCollection"
geojson["features"] = []

for idx, single_hotel in hotels.reset_index().iterrows():
    hotel_entry = {}
    hotel_entry["type"] = "Feature"
    hotel_entry["geometry"] = json.loads(single_hotel.geometry.replace("\'", "\""))
    hotel_entry["properties"] = {}
    hotel_entry["properties"]["h"] = single_hotel.novos_ET
    geojson["features"].append(hotel_entry)

with open(f'../web/public/static/data/hotels.json', 'w') as fp:
    json.dump(geojson, fp, separators=(',', ':'))

In [22]:
hotels.geometry[0]

"{'type': 'MultiLineString', 'coordinates': [[[-8.6224058, 41.1742319], [-8.6224311, 41.1745623], [-8.6224443, 41.1749254]], [[-8.6224443, 41.1749254], [-8.622428, 41.1757767], [-8.6224086, 41.1761959], [-8.6223626, 41.1770772], [-8.6223441, 41.1773783]], [[-8.6224076, 41.1799712], [-8.622421, 41.1789647], [-8.6224679, 41.178134], [-8.622481, 41.1779013]], [[-8.622481, 41.1779013], [-8.622523, 41.177265], [-8.6225697, 41.176201], [-8.6225901, 41.175563]], [[-8.6230011, 41.1708945], [-8.6230529, 41.1710119], [-8.6230709, 41.1711061], [-8.6230705, 41.1712116], [-8.6230229, 41.1713075], [-8.622721, 41.1716731], [-8.6224831, 41.1719383]], [[-8.6224832, 41.1739327], [-8.6224233, 41.1735894], [-8.6223571, 41.1732353], [-8.6223101, 41.1729785]], [[-8.6223101, 41.1729785], [-8.6222687, 41.1727099], [-8.6222773, 41.1725671], [-8.6223183, 41.1723975], [-8.6224389, 41.1721984], [-8.6226464, 41.1719602], [-8.6231442, 41.1713668], [-8.6232266, 41.1712179]], [[-8.6224831, 41.1719383], [-8.6223508, 4